In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import flwr
import torch
import dotenv
import wandb
import os

from torch.optim.lr_scheduler import CosineAnnealingLR

from fl_g13.fl_pytorch import build_fl_dependencies

from fl_g13.fl_pytorch.editing.centralized_mask import get_client_masks
from fl_g13.fl_pytorch.editing.centralized_mask import aggregate_by_sum
from fl_g13.editing.masking import mask_dict_to_list

from fl_g13.fl_pytorch.editing.centralized_mask import load_mask
from fl_g13.fl_pytorch.editing.centralized_mask import save_mask

from fl_g13.fl_pytorch.datasets import reset_partition
from fl_g13.fl_pytorch.client_app import get_client_app
from fl_g13.fl_pytorch.server_app import get_server_app
from flwr.simulation import run_simulation

from fl_g13.architectures import BaseDino
from fl_g13.editing import SparseSGDM

c:\Users\ciovi\Desktop\coding\mldl\fl-g13\.venv\lib\site-packages\dockerpycreds\utils.py:1: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn
2025-06-18 19:36:27.324 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ciovi\Desktop\coding\mldl\fl-g13


In [3]:
dotenv.load_dotenv()

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")

build_fl_dependencies()

# login by key in .env file
WANDB_API_KEY = dotenv.dotenv_values()["WANDB_API_KEY"]
wandb.login(key=WANDB_API_KEY)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Training on cuda
Flower 1.17.0 / PyTorch 2.7.1+cu128
'vision_transformer.py' already exists.
'utils.py' already exists.


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\ciovi\_netrc
wandb: Currently logged in as: tarantino-giovanbattista01 (stefano-gamba-social-politecnico-di-torino) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# FL CONFIG

In [4]:
CHECKPOINT_DIR = dotenv.dotenv_values()["CHECKPOINT_DIR"]

# Model config
## Model Hyper-parameters
head_layers = 3
head_hidden_size = 512
dropout_rate = 0.0
unfreeze_blocks = 1

## Training Hyper-parameters
batch_size = 128
lr = 1e-3
momentum = 0.9
weight_decay = 1e-5
T_max = 8
eta_min = 1e-5

# FL config
K = 100
C = 0.1
J = 8
num_rounds = 30
partition_type = 'shard'

## only for partition_type = 'shard'
num_shards_per_partition = 50

## Server App config
save_every = 5
fraction_fit = C  # Sample of available clients for training
fraction_evaluate = 0.1  # Sample 50% of available clients for evaluation
min_fit_clients = 10  # Never sample less than 10 clients for training
min_evaluate_clients = 5  # Never sample less than 5 clients for evaluation
min_available_clients = 10  # Wait until all 10 clients are available
device = DEVICE
client_batch_size = 16

# Quorum strategy
strategy = 'quorum'
quorum_increment = 7
quorum_update_frequency = 5
initial_quorum = 1

# model editing config
model_editing = True
mask_type = 'local'
sparsity = 0.8
num_calibration_rounds = 3
mask = None

## simulation run config
NUM_CLIENTS = 100
MAX_PARALLEL_CLIENTS = 10

total_rounds = 300
evaluate_each = 5

# Device settings

# When running on GPU, assign an entire GPU for each client
# Refer to Flower framework documentation for more details about Flower simulations
# and how to set up the `backend_config`
if device == "cuda":
    backend_config = {
        "client_resources": {
            "num_cpus": 1, 
            "num_gpus": 1
        }
    }
else:
    backend_config = {
        "client_resources": {
            "num_cpus": 1, 
            "num_gpus": 0
        }
    }

print(f"Training on {device}")

partition_name = 'iid' if partition_type == 'iid' else 'non_iid'
model_save_path = CHECKPOINT_DIR + f"/fl_dino_v4/{partition_name}/{num_shards_per_partition}_{J}"
model_config={
    "head_layers": 3,
    "head_hidden_size": 512,
    "dropout_rate": 0.0,
    "unfreeze_blocks": 0,
}

Training on cuda


In [5]:
from fl_g13.modeling import load_or_create

# Model
model, start_epoch = load_or_create(
    path=model_save_path,
    model_class=BaseDino,
    model_config=None,
    optimizer=None,
    scheduler=None,
    device=device,
    verbose=True,
)
model.to(DEVICE)

unfreeze_blocks = 12
model.unfreeze_blocks(unfreeze_blocks)
# optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)

# Create a dummy mask for SparseSGDM
# Must be done AFTER the model is moved to the device
init_mask = [torch.ones_like(p, device=p.device) for p in model.parameters()]

# Optimizer, scheduler, and loss function
optimizer = SparseSGDM(
    model.parameters(),
    mask=init_mask,
    lr=lr,
    momentum=0.9,
    weight_decay=1e-5
)
criterion = torch.nn.CrossEntropyLoss()
scheduler = CosineAnnealingLR(
    optimizer=optimizer,
    T_max=T_max,
    eta_min=eta_min
)

🔍 Loading checkpoint from /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/fl_dino_v4/non_iid/50_8\fl_fl_baseline_BaseDino_epoch_200_noniid_50_8.pth
📦 Model class in checkpoint: BaseDino
🔧 Model configuration: {'variant': 'dino_vits16', 'dropout_rate': 0.0, 'head_hidden_size': 512, 'head_layers': 3, 'num_classes': 100, 'unfreeze_blocks': 0, 'activation_fn': 'GELU', 'pretrained': True}


Using cache found in C:\Users\ciovi/.cache\torch\hub\facebookresearch_dino_main
Using cache found in C:\Users\ciovi/.cache\torch\hub\facebookresearch_dino_main


➡️ Moved model to device: cuda
✅ Loaded checkpoint from /Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/fl_dino_v4/non_iid/50_8\fl_fl_baseline_BaseDino_epoch_200_noniid_50_8.pth, resuming at epoch 201


In [6]:
return_scores = False

file_name = f'{num_shards_per_partition}_{J}_sum_mask_{mask_type}_{sparsity}_{num_calibration_rounds}.pth'

if os.path.isfile(CHECKPOINT_DIR + f'/masks/{file_name}'):
    print(f'Found {file_name}. Loading mask from memory')
    sum_mask = mask_dict_to_list(model, load_mask(CHECKPOINT_DIR + f'/masks/{file_name}'))
else:
    print(f"Computing sum mask")
    masks, scores, _ = get_client_masks(
        ## config client data set params
        client_partition_type=partition_type,  # 'iid' or 'shard' for non-iid dataset
        client_num_partitions=NUM_CLIENTS,  # equal to number of client
        client_num_shards_per_partition=num_shards_per_partition,
        client_batch_size=client_batch_size,

        ## config get mask params
        mask_model=model,
        mask_sparsity=sparsity,
        mask_type=mask_type,
        mask_rounds=num_calibration_rounds,
        return_scores = return_scores
    )
    sum_mask = aggregate_by_sum(masks)
    save_mask(sum_mask, CHECKPOINT_DIR + f'/masks/{file_name}')
    print(f'Mask stored for future uses at ({CHECKPOINT_DIR + f"/masks/{file_name}"})')
    sum_mask = mask_dict_to_list(model, sum_mask)

print('\nDONE !!')

Computing sum mask


Clients mask: 100%|██████████| 100/100 [07:35<00:00,  4.55s/it]


Mask stored for future uses at (/Users/ciovi/Desktop/coding/mldl/fl-g13/checkpoints/masks/50_8_sum_mask_local_0.8_3.pth)

DONE !!


# Simulation

In [7]:
reset_partition()

checkpoint_dir = CHECKPOINT_DIR + f"/fl_dino_v4/{partition_name}/{num_shards_per_partition}_{J}_dq"
os.makedirs(checkpoint_dir, exist_ok=True)

## Wandb config
use_wandb = True
name = f"DQ_{num_shards_per_partition}_{J}_{mask_type}_{sparsity}_{num_calibration_rounds}"

wandb_config = {
    # wandb param
    'name': name,
    'project_name': "Dynamic-Quorum",
    
    # model config
    "lr": lr,
    "momentum": momentum,
    
    # FL config
    'K': K,
    'C': C,
    'J': J,
    'partition_type': partition_type,
    "fraction_fit": fraction_fit,
    
    # model editing config
    'mask_type': mask_type,
    'sparsity': sparsity,
    'num_calibration_rounds': num_calibration_rounds,
    
    # strategy config
    'initial_quorum': initial_quorum,
    'quorum_update_frequency': quorum_update_frequency,
    'quorum_increment': quorum_increment
}

client = get_client_app(
        model=model,
        optimizer=optimizer,
        criterion=criterion,
        device=device,
        partition_type=partition_type,
        local_epochs=1,
        local_steps=J,
        batch_size=batch_size,
        num_shards_per_partition=num_shards_per_partition,
        scheduler=scheduler,
        verbose=0,
        mask=mask,
        model_editing_batch_size=client_batch_size,
        mask_func=None,
        strategy=strategy,
    )

compute_round = total_rounds + 1 - start_epoch

server = get_server_app(
    global_mask = sum_mask,
    num_total_clients = NUM_CLIENTS,
    quorum_increment = quorum_increment,
    quorum_update_frequency = quorum_update_frequency,
    initial_quorum = initial_quorum,
    
    checkpoint_dir=checkpoint_dir,
    model_class=model,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    num_rounds=compute_round,
    fraction_fit=fraction_fit,
    fraction_evaluate=fraction_evaluate,
    min_fit_clients=min_fit_clients,
    min_evaluate_clients=min_evaluate_clients,
    min_available_clients=min_available_clients,
    device = device,
    use_wandb=use_wandb,
    wandb_config = wandb_config,
    save_every=save_every,
    prefix = name,
    evaluate_each=evaluate_each,
    model = model,
    start_epoch=start_epoch,
    strategy=strategy
)

In [8]:
try:
    run_simulation(
        server_app=server,
        client_app=client,
        num_supernodes=NUM_CLIENTS,
        backend_config=backend_config
    )
except:
    pass

wandb.finish()

[Server] Server on device: cuda:0
[Server] CUDA available in client: True
Using strategy 'DynmicQuorum'


c:\Users\ciovi\Desktop\coding\mldl\fl-g13\.venv\lib\site-packages\wandb\analytics\sentry.py:259: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}  # noqa


INFO :      Starting Flower ServerApp, config: num_rounds=100, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


[Server Eval Round 0] Model device: cuda:0
[Server Eval Round 0] CUDA available in server eval: True


Eval progress: 100%|██████████| 313/313 [00:22<00:00, 13.84batch/s]
INFO :      [Round 0] Centralized Evaluation - Loss: 1.2866, Metrics: {'centralized_accuracy': 0.632}
INFO :      initial parameters (loss, other metrics): 1.2865915495557145, {'centralized_accuracy': 0.632}
INFO :      
INFO :      [ROUND 1]
INFO :      [Round 1] Generated global mask with sparsity: 0.7309
INFO :      configure_fit: strategy sampled 10 clients (out of 100)
(ClientAppActor pid=15980) 2025-06-18 19:44:35.391 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ciovi\Desktop\coding\mldl\fl-g13
(ClientAppActor pid=15980) c:\Users\ciovi\Desktop\coding\mldl\fl-g13\.venv\lib\site-packages\dockerpycreds\utils.py:1: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
(ClientAppActor pid=15980)   import distutils.spawn


...

(ClientAppActor pid=15980) [Client] Client on device: cuda:0
(ClientAppActor pid=15980) [Client] CUDA available in client: True


Eval progress: 100%|██████████| 1/1 [00:00<00:00,  3.90batch/s]
(ClientAppActor pid=15980) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
Eval progress:   0%|          | 0/1 [00:00<?, ?batch/s]


(ClientAppActor pid=15980) [Client] Client on device: cuda:0
(ClientAppActor pid=15980) [Client] CUDA available in client: True


INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 100 round(s) in 6221.21s
INFO :      	History (loss, distributed):
INFO :      		round 1: 1.0921281158924103
INFO :      		round 2: 1.0142948985099793
INFO :      		round 3: 1.0972393810749055
INFO :      		round 4: 1.0775072872638702
INFO :      		round 5: 1.0567696511745452
INFO :      		round 6: 0.8833150446414948
INFO :      		round 7: 0.9882081985473633
INFO :      		round 8: 0.8966444313526154
INFO :      		round 9: 0.9764277458190918
INFO :      		round 10: 0.9162006974220276
INFO :      		round 11: 1.0161004543304444
INFO :      		round 12: 0.9016785025596619
INFO :      		round 13: 0.9291084587574006
INFO :      		round 14: 0.9500333547592164
INFO :      		round 15: 0.80823974609375
INFO :      		round 16: 0.8770976781845092
INFO :      		round 17: 0.8424881100654602
INFO :      		round 18: 0.8667359232902527
INFO :      		round 19: 0.907150274515152
I

centralized_accuracy,▁▃▄▄▅▆▆▆▆▇▇▇▇███████
centralized_eval_loss,█▆▅▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁
decentralized_avg_eval_accuracy,▁▂▄▅▄▃▅▆▄▆▄▅▄▅▅▆▇▇▅▇▇█▅▇▆▆▇▇▆▇▆▇▇█▇▇▆▆▆█
decentralized_avg_eval_loss,█▅▆▇▅▄▅▄▄▄▅▃▄▅▄▂▂▂▃▃▃▂▃▃▂▃▂▂▂▁▂▂▃▂▃▂▂▃▁▃
decentralized_avg_train_accuracy,▁▃▃▃▄▅▄▄▅▅▆▆▄▆▇▅▇▇▆▆▆▇▇▇▆▇▆▇▆█▆▅▆▇▇▆▇▆▇▇
decentralized_avg_train_loss,█▅▅▅▅▃▄▂▄▂▃▃▂▂▃▃▂▂▂▂▃▂▂▂▁▁▂▁▂▁▁▃▂▁▂▂▃▂▁▂
mask_sparsity,▁▁▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▇▇████████
quorum,▁▁▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇███████████████
centralized_accuracy,0.7824
centralized_eval_loss,0.71683
decentralized_avg_eval_accuracy,0.775
